In [1]:
import pulp as p
import numpy as np
import math
import pandas as pd

from scipy import stats

import random

import math 

In [2]:
def prod_run(start, demandqty, setupcost, holdingcost):
    mincost = setupcost 
    cost = mincost
    for i in range(start,len(demandqty)):
        if i == (len(demandqty)-1):
            return i
            break
        period = len(demandqty[start:i+1])
        cost = (cost*period + demandqty[i+1]*holdingcost*(i+1-start)) / (period+1)
        if cost > mincost:
            return i
        elif cost <= mincost:
            mincost = cost
    
def silver_meal(demandqty, initialinv, setupcost, holdingcost):
    endinginv = np.zeros([len(demandqty)])
    production = np.zeros([len(demandqty)])
    i = 0
    while i < len(demandqty):
        runs = prod_run(i, demandqty, setupcost, holdingcost)
        production[i] = sum([demandqty[j] for j in range(i, runs+1)])
        i = runs+1
    for i in range(len(demandqty)):
        if i == 0:
            endinginv[i] = initialinv + production[i] - demandqty[i]
        else:
            endinginv[i] = endinginv[i-1] + production[i] - demandqty[i]
            
    totalholdingcost = sum([endinginv[i] for i in range(len(endinginv))])*holdingcost
    setup_on = production>0
    setup_on = setup_on.astype(int)
    totalsetupcost = sum(setup_on[i] for i in range(len(production)))*setupcost
    totalcost = totalholdingcost + totalsetupcost
    
    print('Production: ', production)
    print('Totalholdingcost: ', totalholdingcost)
    print('Totalsetupcost: ', totalsetupcost)
    print('Totalcost: ', totalcost)
    #return production, totalholdingcost, totalsetupcost, totalcost


<img src = "w3lect1.png">

In [165]:
timeperiod = [1,2,3,4,5,6,7,8,9,10,11,12]
timeperiod_with_0 = [0,1,2,3,4,5,6,7,8,9,10,11,12]

demandqty = np.array([200,150,100,50,50,100,150,200,200,250,300,250])


setupcost = 500
holdingcost = 1
capacity = 99999

prob = p.LpProblem('Production', p.LpMinimize)

prodqty = p.LpVariable.dicts("Production_Qty", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

prodopen = p.LpVariable.dicts("Production_Open", timeperiod, cat = 'Binary')

endinventory = p.LpVariable.dicts("Inventory_Qty", timeperiod_with_0, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

demandqtydict = dict(zip(timeperiod, demandqty))


prob += p.lpSum([prodopen[t]*setupcost for t in timeperiod]) + p.lpSum([endinventory[t]*holdingcost for t in timeperiod])

prob += endinventory[0] == 0 #starting inventory of constraint, in this case it is 0

for t in timeperiod:
    
    prob += prodqty[t] - demandqtydict[t] - endinventory[t] + endinventory[t-1] == 0 #conservation of flow constraint
    
    prob += prodqty[t] - demandqty.sum()*prodopen[t] <= 0 #linkingconstraint if any qty is produced then setup is open

    prob += prodqty[t] <= capacity #production in each time period is less than capacity
    
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

minimum cost:  3750.0
Solution Status: Optimal
Inventory_Qty_0  =  0.0
Inventory_Qty_1  =  350.0
Inventory_Qty_10  =  0.0
Inventory_Qty_11  =  250.0
Inventory_Qty_12  =  0.0
Inventory_Qty_2  =  200.0
Inventory_Qty_3  =  100.0
Inventory_Qty_4  =  50.0
Inventory_Qty_5  =  0.0
Inventory_Qty_6  =  350.0
Inventory_Qty_7  =  200.0
Inventory_Qty_8  =  0.0
Inventory_Qty_9  =  250.0
Production_Open_1  =  1.0
Production_Open_10  =  0.0
Production_Open_11  =  1.0
Production_Open_12  =  0.0
Production_Open_2  =  0.0
Production_Open_3  =  0.0
Production_Open_4  =  0.0
Production_Open_5  =  0.0
Production_Open_6  =  1.0
Production_Open_7  =  0.0
Production_Open_8  =  0.0
Production_Open_9  =  1.0
Production_Qty_1  =  550.0
Production_Qty_10  =  0.0
Production_Qty_11  =  550.0
Production_Qty_12  =  0.0
Production_Qty_2  =  0.0
Production_Qty_3  =  0.0
Production_Qty_4  =  0.0
Production_Qty_5  =  0.0
Production_Qty_6  =  450.0
Production_Qty_7  =  0.0
Production_Qty_8  =  0.0
Production_Qty_9  =  450

<img src = "w3pp1.png">


Formulate a MILP-spreadsheet to find the optimal production plan.

In the optimal solution, what is the total cost to produce the 10 week demand for NOX12K678P? Please give your answer in dollars without commas or currency symbols.

In [166]:
timeperiod = [1,2,3,4,5,6,7,8,9,10]
timeperiod_with_0 = [0,1,2,3,4,5,6,7,8,9,10]

demandqty = np.array([200,180,220,170,160,170,180,190,220,210])


setupcost = 1200
holdingcost = 1
capacity = 99999

prob = p.LpProblem('Production', p.LpMinimize)

prodqty = p.LpVariable.dicts("Production_Qty", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

prodopen = p.LpVariable.dicts("Production_Open", timeperiod, cat = 'Binary')

endinventory = p.LpVariable.dicts("Inventory_Qty", timeperiod_with_0, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

demandqtydict = dict(zip(timeperiod, demandqty))


prob += p.lpSum([prodopen[t]*setupcost for t in timeperiod]) + p.lpSum([endinventory[t]*holdingcost for t in timeperiod])

prob += endinventory[0] == 300 #starting inventory of 300

for t in timeperiod:
    
    prob += prodqty[t] - demandqtydict[t] - endinventory[t] + endinventory[t-1] == 0 #conservation of flow constraint
    
    prob += prodqty[t] - demandqty.sum()*prodopen[t] <= 0 #linkingconstraint if any qty is produced then setup is open
    
    prob += prodqty[t] <= capacity #production in each time period is less than capacity
    
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

minimum cost:  5430.0
Solution Status: Optimal
Inventory_Qty_0  =  300.0
Inventory_Qty_1  =  100.0
Inventory_Qty_10  =  0.0
Inventory_Qty_2  =  390.0
Inventory_Qty_3  =  170.0
Inventory_Qty_4  =  0.0
Inventory_Qty_5  =  350.0
Inventory_Qty_6  =  180.0
Inventory_Qty_7  =  0.0
Inventory_Qty_8  =  430.0
Inventory_Qty_9  =  210.0
Production_Open_1  =  0.0
Production_Open_10  =  0.0
Production_Open_2  =  1.0
Production_Open_3  =  0.0
Production_Open_4  =  0.0
Production_Open_5  =  1.0
Production_Open_6  =  0.0
Production_Open_7  =  0.0
Production_Open_8  =  1.0
Production_Open_9  =  0.0
Production_Qty_1  =  0.0
Production_Qty_10  =  0.0
Production_Qty_2  =  470.0
Production_Qty_3  =  0.0
Production_Qty_4  =  0.0
Production_Qty_5  =  510.0
Production_Qty_6  =  0.0
Production_Qty_7  =  0.0
Production_Qty_8  =  620.0
Production_Qty_9  =  0.0


<h3>Practice Problem 2: Sodaco</h3>
    
Sodaco runs a plant that makes and bottles private label soft drinks, sold directly to retailers. One client has recently placed an order for one of the SKU's with the following delivery schedule over the next ten weeks (one period=one week):

<img src = "w3pp2.png">

The order is of one-time character - there are no promises for future orders from the same client.

Sodaco needs your help to create a production plan that minimizes the total costs. Costs are incurred both from holding inventory of finished goods and from changeovers in the production line. Sodaco estimates these costs according to the following:

Inventory holding costs: $0.1 per item and week

Production setup costs: $800 per production run

Any products delivered in the same week as manufactured incur no holding costs. Holding costs are then incurred for every week after the manufacturing week. Capacity is not an issue at this point.



In [167]:
timeperiod = [1,2,3,4,5,6,7,8,9,10]
timeperiod_with_0 = [0,1,2,3,4,5,6,7,8,9,10]

demandqty = np.array([3000,3100,3200,3100,2900,2800,2800,2900,2800,3000])


setupcost = 800
holdingcost = 0.1
capacity = 999999999

prob = p.LpProblem('Production', p.LpMinimize)

prodqty = p.LpVariable.dicts("Production_Qty", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

prodopen = p.LpVariable.dicts("Production_Open", timeperiod, cat = 'Binary')

endinventory = p.LpVariable.dicts("Inventory_Qty", timeperiod_with_0, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

demandqtydict = dict(zip(timeperiod, demandqty))


prob += p.lpSum([prodopen[t]*setupcost for t in timeperiod]) + p.lpSum([endinventory[t]*holdingcost for t in timeperiod])

prob += endinventory[0] == 0 #starting inventory constraint

for t in timeperiod:
    
    prob += prodqty[t] - demandqtydict[t] - endinventory[t] + endinventory[t-1] == 0 #conservation of flow constraint
    
    prob += prodqty[t] - demandqty.sum()*prodopen[t] <= 0 #linkingconstraint if any qty is produced then setup is open
    
    prob += prodqty[t] <= capacity #production in each time period is less than capacity
    
    
prob.solve()


print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

minimum cost:  5490.0
Solution Status: Optimal
Inventory_Qty_0  =  0.0
Inventory_Qty_1  =  3100.0
Inventory_Qty_10  =  0.0
Inventory_Qty_2  =  0.0
Inventory_Qty_3  =  3100.0
Inventory_Qty_4  =  0.0
Inventory_Qty_5  =  2800.0
Inventory_Qty_6  =  0.0
Inventory_Qty_7  =  2900.0
Inventory_Qty_8  =  0.0
Inventory_Qty_9  =  3000.0
Production_Open_1  =  1.0
Production_Open_10  =  0.0
Production_Open_2  =  0.0
Production_Open_3  =  1.0
Production_Open_4  =  0.0
Production_Open_5  =  1.0
Production_Open_6  =  0.0
Production_Open_7  =  1.0
Production_Open_8  =  0.0
Production_Open_9  =  1.0
Production_Qty_1  =  6100.0
Production_Qty_10  =  0.0
Production_Qty_2  =  0.0
Production_Qty_3  =  6300.0
Production_Qty_4  =  0.0
Production_Qty_5  =  5700.0
Production_Qty_6  =  0.0
Production_Qty_7  =  5700.0
Production_Qty_8  =  0.0
Production_Qty_9  =  5800.0


<h3>Practice Problem 3: Custom Glass Inc.</h3>

Custom Glass Inc. makes custom-made windows and window frames for all types of buildings. They have just received an order from a client to make and deliver 1,340 framed windows. The windows, which are identical to one another, are to be delivered over the course of six months, according to the following schedule:

January: 300 windows

February: 280 windows

March: 200 windows

April: 190 windows

May: 160 windows

June: 210 windows

To avoid tying up too much working capital, the production manager of Custom Glass, Mr. Dumpty, is hesitant to produce all 1,340 windows at once. Not only would this tie up capital, it would also lead to a high risk obsolescence due to the rough handling in the warehouse. However, splitting up the order into several batches is also costly. For every batch they make there will be an initial setup cost due to equipment changeover and scrap from the "burn-in" period when the equipment is being fine-tuned.

Mr. Dumpty estimates the unit holding cost to be 2dollars per month and the setup cost to be 1,000dollars per batch. If the windows are shipped within the same month, they do not incur any holding costs, but for future periods they do. Custom Glass can easily produce all windows in a day, so capacity is not an issue. The factory closes at night, so you can assume there is a setup cost for the first batch every day, independently of whether it was the last batch the day before or not.

Mr. Dumpty now asks for your help in making a production plan for the order.

Part 7

You want to explore if maybe a more advanced heuristic would provide a better result, and decide to try the Silver-Meal heuristic.

Using the Silver-Meal heuristic, what is the production quantity for each of the six months? Answer in integer units without commas.

Part 8

What is the total cost to produce the ordered windows when using the Silver-Meal heuristic in Part 7? Answer in integer dollars without commas or currency symbols.

In [282]:
demandqty = np.array([300,280,200,190,160,210])
holdingcost = 2
setupcost = 1000
initialinv = 0

silver_meal(demandqty, initialinv, setupcost, holdingcost)    

Production:  [580.   0. 550.   0.   0. 210.]
Totalholdingcost:  1580.0
Totalsetupcost:  3000
Totalcost:  4580.0


Part 9

Mr. Dumpty is still skeptical. He understands that the heuristics you presented are just heuristics and may be far from the optimal solution. Now he wants to know what the actual optimal schedule would look like.

Create a spreadsheet model to find the optimal production schedule. Based on this model, what is the production quantity for each of the six months? Answer in integer units without commas.

Part 10

What is the total cost to produce the ordered windows in the optimal solution? Answer in integer dollars without commas or currency symbols.

In [283]:
timeperiod = [1,2,3,4,5,6]
timeperiod_with_0 = [0,1,2,3,4,5,6]

demandqty = np.array([300,280,200,190,160,210])


setupcost = 1000
holdingcost = 2
capacity = 999999999

prob = p.LpProblem('Production', p.LpMinimize)

prodqty = p.LpVariable.dicts("Production_Qty", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

prodopen = p.LpVariable.dicts("Production_Open", timeperiod, cat = 'Binary')

endinventory = p.LpVariable.dicts("Inventory_Qty", timeperiod_with_0, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

demandqtydict = dict(zip(timeperiod, demandqty))


prob += p.lpSum([prodopen[t]*setupcost for t in timeperiod]) + p.lpSum([endinventory[t]*holdingcost for t in timeperiod])

prob += endinventory[0] == 0 #starting inventory constraint

for t in timeperiod:
    
    prob += prodqty[t] - demandqtydict[t] - endinventory[t] + endinventory[t-1] == 0 #conservation of flow constraint
    
    prob += prodqty[t] - demandqty.sum()*prodopen[t] <= 0 #linkingconstraint if any qty is produced then setup is open
    
    prob += prodqty[t] <= capacity #production in each time period is less than capacity
    
    
prob.solve()


print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

minimum cost:  4360.0
Solution Status: Optimal
Inventory_Qty_0  =  0.0
Inventory_Qty_1  =  280.0
Inventory_Qty_2  =  0.0
Inventory_Qty_3  =  190.0
Inventory_Qty_4  =  0.0
Inventory_Qty_5  =  210.0
Inventory_Qty_6  =  0.0
Production_Open_1  =  1.0
Production_Open_2  =  0.0
Production_Open_3  =  1.0
Production_Open_4  =  0.0
Production_Open_5  =  1.0
Production_Open_6  =  0.0
Production_Qty_1  =  580.0
Production_Qty_2  =  0.0
Production_Qty_3  =  390.0
Production_Qty_4  =  0.0
Production_Qty_5  =  370.0
Production_Qty_6  =  0.0


<h1>Graded Assignment 1: Gold Mit Seals</h1>
    
There is a retailer called "The Coop", which carries a wide variety of products with MIT's name and logo: shirts, caps, keychains, pens... you name it! Anything from golf balls and teddy bears to pocket protectors and jewelry. With a store inside MIT's Student Center, and another one in the popular Kendall Square, across the street from MIT campus, The Coop is a favorite stop for casual MIT visitors, current students and their relatives, and nostalgic alumni.

The Coop carries a line of products that feature MIT's official seal cast in a jewelry-grade steel. The same seal (about the size of a coin) is used in multiple finished goods, such as necklaces, tie pins, cufflinks, and paperweights. The Coop is considering introducing in 2018 a new line of products featuring MIT's seal cast in 18 karat gold. These seals would be used in upscale jewelry, and as an ornament in the diploma frames that sell especially during commencement season.

Based on their experience with similar products in the past, The Coop has projected the following demand for gold MIT seals for each month of 2018.

January	260

February	60

March	120

April	100

May	400

June	1100

July	360

August	280

September	120

October	100

November	200

December	500

The seals would be produced, under license, by Seventh Seal, a specialty manufacturer of commemorative seals cast in precious metals, located in Syracuse, New York. Seventh Seal has offered to produce the gold MIT seals exclusively for The Coop. Seventh Seal has explained to The Coop that there is a cost to setting up the equipment to produce the gold MIT seal. Because of this, Seventh Seal will charge The Coop a set-up cost of 1026.0dollars every time the MIT seal is produced.

Because of the high set-up cost, one of the managers at The Coop proposes doing all the seals that will be needed for 2018 in a one-time production batch. "This will save us a lot of money in set-up costs", he says.

Another manager, however, warns that - because the gold seals are expensive - there is an associated holding cost. The Coop estimates that the holding cost will be about 0.76dollars per seal per month (the dot is a decimal mark).

Because of the high holding costs, that manager proposes doing monthly batches in the amount of seals that will be required that month, according to the demand projection. "This will save us a lot of money in holding costs", she says.

Since they know that you are taking SC2x, they ask for your help in evaluating these two alternatives.

Part 7

What is the total cost (e.g. the sum of set-up costs and holding costs) of producing the gold seals using a Silver-Meal algorithm?




In [11]:
demandqty = np.array([260,60,120,100,400,1100,360,280,120,100,200,500])
holdingcost = 0.76
setupcost = 1026
initialinv = 0

silver_meal(demandqty, initialinv, setupcost, holdingcost)  

Production:  [ 540.    0.    0.    0. 2360.    0.    0.    0.    0.    0.  700.    0.]
Totalholdingcost:  3602.4
Totalsetupcost:  3078
Totalcost:  6680.4


Part 8

What is the total cost (e.g. the sum of set-up costs and holding costs) of producing the gold seals using an optimal solution?



In [4]:
timeperiod = [1,2,3,4,5,6,7,8,9,10,11,12]
timeperiod_with_0 = [0,1,2,3,4,5,6,7,8,9,10,11,12]

demandqty = np.array([260,60,120,100,400,1100,360,280,120,100,200,500])


setupcost = 1026
holdingcost = 0.76
capacity = 999999999

prob = p.LpProblem('Production', p.LpMinimize)

prodqty = p.LpVariable.dicts("Production_Qty", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

prodopen = p.LpVariable.dicts("Production_Open", timeperiod, cat = 'Binary')

endinventory = p.LpVariable.dicts("Inventory_Qty", timeperiod_with_0, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

demandqtydict = dict(zip(timeperiod, demandqty))


prob += p.lpSum([prodopen[t]*setupcost for t in timeperiod]) + p.lpSum([endinventory[t]*holdingcost for t in timeperiod])

prob += endinventory[0] == 0 #starting inventory constraint

for t in timeperiod:
    
    prob += prodqty[t] - demandqtydict[t] - endinventory[t] + endinventory[t-1] == 0 #conservation of flow constraint
    
    prob += prodqty[t] - demandqty.sum()*prodopen[t] <= 0 #linkingconstraint if any qty is produced then setup is open
    
    prob += prodqty[t] <= capacity #production in each time period is less than capacity
    
    
prob.solve()


print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

minimum cost:  6216.8
Solution Status: Optimal
Inventory_Qty_0  =  0.0
Inventory_Qty_1  =  280.0
Inventory_Qty_10  =  0.0
Inventory_Qty_11  =  500.0
Inventory_Qty_12  =  0.0
Inventory_Qty_2  =  220.0
Inventory_Qty_3  =  100.0
Inventory_Qty_4  =  0.0
Inventory_Qty_5  =  0.0
Inventory_Qty_6  =  860.0
Inventory_Qty_7  =  500.0
Inventory_Qty_8  =  220.0
Inventory_Qty_9  =  100.0
Production_Open_1  =  1.0
Production_Open_10  =  0.0
Production_Open_11  =  1.0
Production_Open_12  =  0.0
Production_Open_2  =  0.0
Production_Open_3  =  0.0
Production_Open_4  =  0.0
Production_Open_5  =  1.0
Production_Open_6  =  1.0
Production_Open_7  =  0.0
Production_Open_8  =  0.0
Production_Open_9  =  0.0
Production_Qty_1  =  540.0
Production_Qty_10  =  0.0
Production_Qty_11  =  700.0
Production_Qty_12  =  0.0
Production_Qty_2  =  0.0
Production_Qty_3  =  0.0
Production_Qty_4  =  0.0
Production_Qty_5  =  400.0
Production_Qty_6  =  1960.0
Production_Qty_7  =  0.0
Production_Qty_8  =  0.0
Production_Qty_9  = 

<h3>MicroMasters Practice Problem 1: Bluth Construction</h3>

The Bluth Construction company makes garden sheds for consumers. For the coming 12 months they have the following estimated demand for one of their big sellers (1 period = 1 month):

<img src = "w3supp11.png">

They have estimated the cost of holding inventory to be 1dollar per finished shed per month. These costs are not incurred during the month of production, but are for future months. The setup cost per run is 600dollar. Capacity is not an issue.

The production manager, Michael Bluth, has had a lot of things on his plate lately and realizes he needs your assistance in planning the production.

<b>Part 1</b>

You decide to try the Silver-Meal heuristic.

Using the Silver-Meal heuristic, what is the total cost of producing the best seller garden shed? Please give your answer in dollars without commas or currency symbols.


In [3]:
demandqty = np.array([400,410,390,380,390,400,410,420,410,390,375,400])
holdingcost = 1
setupcost = 600
initialinv = 0

silver_meal(demandqty, initialinv, setupcost, holdingcost)  

Production:  [810.   0. 770.   0. 790.   0. 830.   0. 800.   0. 775.   0.]
Totalholdingcost:  2400.0
Totalsetupcost:  3600
Totalcost:  6000.0


<b>Part 2</b>

Create a spreadsheet and find the optimal production plan.

How far from the optimal solution is your heuristic solution from Part 1? Please give your answer in dollars without commas or currency symbols. For instance if your solution in Part 1 is 5,800 and the optimal solution from your model is 5,500, answer 5,800-5,500=300.

In [4]:
timeperiod = [1,2,3,4,5,6,7,8,9,10,11,12]
timeperiod_with_0 = [0,1,2,3,4,5,6,7,8,9,10,11,12]

demandqty = np.array([400,410,390,380,390,400,410,420,410,390,375,400])


setupcost = 600
holdingcost = 1
capacity = 999999999

prob = p.LpProblem('Production', p.LpMinimize)

prodqty = p.LpVariable.dicts("Production_Qty", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

prodopen = p.LpVariable.dicts("Production_Open", timeperiod, cat = 'Binary')

endinventory = p.LpVariable.dicts("Inventory_Qty", timeperiod_with_0, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

demandqtydict = dict(zip(timeperiod, demandqty))


prob += p.lpSum([prodopen[t]*setupcost for t in timeperiod]) + p.lpSum([endinventory[t]*holdingcost for t in timeperiod])

prob += endinventory[0] == 0 #starting inventory constraint

for t in timeperiod:
    
    prob += prodqty[t] - demandqtydict[t] - endinventory[t] + endinventory[t-1] == 0 #conservation of flow constraint
    
    prob += prodqty[t] - demandqty.sum()*prodopen[t] <= 0 #linkingconstraint if any qty is produced then setup is open
    
    prob += prodqty[t] <= capacity #production in each time period is less than capacity
    
    
prob.solve()


print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

minimum cost:  6000.0
Solution Status: Optimal
Inventory_Qty_0  =  0.0
Inventory_Qty_1  =  410.0
Inventory_Qty_10  =  0.0
Inventory_Qty_11  =  400.0
Inventory_Qty_12  =  0.0
Inventory_Qty_2  =  0.0
Inventory_Qty_3  =  380.0
Inventory_Qty_4  =  0.0
Inventory_Qty_5  =  400.0
Inventory_Qty_6  =  0.0
Inventory_Qty_7  =  420.0
Inventory_Qty_8  =  0.0
Inventory_Qty_9  =  390.0
Production_Open_1  =  1.0
Production_Open_10  =  0.0
Production_Open_11  =  1.0
Production_Open_12  =  0.0
Production_Open_2  =  0.0
Production_Open_3  =  1.0
Production_Open_4  =  0.0
Production_Open_5  =  1.0
Production_Open_6  =  0.0
Production_Open_7  =  1.0
Production_Open_8  =  0.0
Production_Open_9  =  1.0
Production_Qty_1  =  810.0
Production_Qty_10  =  0.0
Production_Qty_11  =  775.0
Production_Qty_12  =  0.0
Production_Qty_2  =  0.0
Production_Qty_3  =  770.0
Production_Qty_4  =  0.0
Production_Qty_5  =  790.0
Production_Qty_6  =  0.0
Production_Qty_7  =  830.0
Production_Qty_8  =  0.0
Production_Qty_9  =  80

<b>Part 3</b>

Suppose the setup costs were instead 1,200dollars per run. Solve the problem using the SM heuristic as well as using your model.

How far from the optimal solution is your heuristic solution ? Please give your answer in dollars without commas or currency symbols. For instance if your heuristic solution is 5,800 and the optimal solution from your model is 5,500, answer 5,800-5,500=300.

In [12]:
demandqty = np.array([400,410,390,380,390,400,410,420,410,390,375,400])
holdingcost = 1
setupcost = 1200
initialinv = 0
print('Optimal Cost Using Silver Meal=')
silver_meal(demandqty, initialinv, setupcost, holdingcost) 
print('\nOptimal Cost Using MILP=')


timeperiod = [1,2,3,4,5,6,7,8,9,10,11,12]
timeperiod_with_0 = [0,1,2,3,4,5,6,7,8,9,10,11,12]

demandqty = np.array([400,410,390,380,390,400,410,420,410,390,375,400])


setupcost = 1200
holdingcost = 1
capacity = 999999999

prob = p.LpProblem('Production', p.LpMinimize)

prodqty = p.LpVariable.dicts("Production_Qty", timeperiod, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

prodopen = p.LpVariable.dicts("Production_Open", timeperiod, cat = 'Binary')

endinventory = p.LpVariable.dicts("Inventory_Qty", timeperiod_with_0, lowBound = 0, cat = 'Continuous') #choose continuous so as to not enforce integer constraint

demandqtydict = dict(zip(timeperiod, demandqty))


prob += p.lpSum([prodopen[t]*setupcost for t in timeperiod]) + p.lpSum([endinventory[t]*holdingcost for t in timeperiod])

prob += endinventory[0] == 0 #starting inventory constraint

for t in timeperiod:
    
    prob += prodqty[t] - demandqtydict[t] - endinventory[t] + endinventory[t-1] == 0 #conservation of flow constraint
    
    prob += prodqty[t] - demandqty.sum()*prodopen[t] <= 0 #linkingconstraint if any qty is produced then setup is open
    
    prob += prodqty[t] <= capacity #production in each time period is less than capacity
    
    
prob.solve()


print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

Optimal Cost Using Silver Meal=
Production:  [1200.    0.    0.  770.    0.  810.    0. 1220.    0.    0.  775.    0.]
Totalholdingcost:  3580.0
Totalsetupcost:  6000
Totalcost:  9580.0

Optimal Cost Using MILP=
minimum cost:  9565.0
Solution Status: Optimal
Inventory_Qty_0  =  0.0
Inventory_Qty_1  =  410.0
Inventory_Qty_10  =  775.0
Inventory_Qty_11  =  400.0
Inventory_Qty_12  =  0.0
Inventory_Qty_2  =  0.0
Inventory_Qty_3  =  770.0
Inventory_Qty_4  =  390.0
Inventory_Qty_5  =  0.0
Inventory_Qty_6  =  410.0
Inventory_Qty_7  =  0.0
Inventory_Qty_8  =  410.0
Inventory_Qty_9  =  0.0
Production_Open_1  =  1.0
Production_Open_10  =  1.0
Production_Open_11  =  0.0
Production_Open_12  =  0.0
Production_Open_2  =  0.0
Production_Open_3  =  1.0
Production_Open_4  =  0.0
Production_Open_5  =  0.0
Production_Open_6  =  1.0
Production_Open_7  =  0.0
Production_Open_8  =  1.0
Production_Open_9  =  0.0
Production_Qty_1  =  810.0
Production_Qty_10  =  1165.0
Production_Qty_11  =  0.0
Production_Qty_

<h3>Midterm Question4 part 1</h3>

SiNRi Healthcare is a leading manufacturing company that produces smart, easy and mobile medical devices. One of their famous products SKU HU16 is a mobile ECG touch-based device. This device can be plugged to any smartphone to view the ECG reading using their specialised app. The app also has features to record and share the result.

Sara Hall is the production team leader of this product line. She needs to plan a practical approach in getting the products out on time. She received the demand estimate for the next quarter as follows:

<img src = 'midterm7.png'>

Currently there are 200 devices in-stock (in week 0). If the products produced in a week are not shipped in the same week, there is a holding cost of 1dollar per device per week. It costs 800dollar to setup and run a batch. There is no limit on the capacity of devices that can be produced.

Question 3

Sara thinks she needs to further dive deep to see if there is a better approach in planning the production runs. She decides to try the silver-meal method.

How many devices would be produced in week 3?

How many devices would be produced in week 7?

In [6]:
netdemandqty = np.array([102,308,324,326,363,386,394,417,440,442,476,494])
holdingcost = 1
setupcost = 800
initialinv = 0

silver_meal(netdemandqty, initialinv, setupcost, holdingcost)    

Production:  [410.   0. 650.   0. 749.   0. 811.   0. 882.   0. 970.   0.]
Totalholdingcost:  2373.0
Totalsetupcost:  4800
Totalcost:  7173.0
